In [1]:
import sys
from PyQt5.QtWidgets import *
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

import numpy as np
import pandas as pd
import scipy.optimize
from sympy import exp
from scipy import optimize
from sympy import *
from numpy import *

import matplotlib.pyplot as plt
import scipy.interpolate
from scipy import interpolate

from scipy.optimize import fmin

YTMs = [0.01508, 0.01622, 0.01778, 0.01904, 0.02078, 0.02145, 0.0222, 0.02267, 0.0246, 0.02505, 0.02632, 0.02711 ]
Maturity = [0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5, 7, 10]

Bond_data = {
    'Maturity' :["%s" %Maturity[0],"%s" %Maturity[1],"%s" %Maturity[2],"%s" %Maturity[3],"%s" %Maturity[4],"%s" %Maturity[5],"%s" %Maturity[6],"%s" %Maturity[7],"%s" %Maturity[8],"%s" %Maturity[9],"%s" %Maturity[10],"%s" %Maturity[11]],
    'YTMs' : ["%s" %YTMs[0],"%s" %YTMs[1],"%s" %YTMs[2],"%s" %YTMs[3],"%s" %YTMs[4],"%s" %YTMs[5],"%s" %YTMs[6],"%s" %YTMs[7],"%s" %YTMs[8],"%s" %YTMs[9],"%s" %YTMs[10],"%s" %YTMs[11]]
    }
column_idx_lookup = {'Maturity' : 0 , 'YTMs' : 1 }


column_idx_lookup_2 = {'Zerorates' : 2}


###########  Bootstrap 코드 ###########
        
number = 1 
        #Maturity = array(data.loc[0,'3월':])

        #a = pd.DataFrame()
        #a['mat'] = Maturity

B = [] 
for i in range(1,41) :
    B.append(0.25*i)
b = pd.DataFrame(B)
C = []
for i in range(1,121) :
    C.append(i)
c = pd.DataFrame(C)


for r in range(1,number+1) :
            #YTMs = data.loc[r,'3월':]/100
    zeros = np.zeros(40)
    tzeros = np.zeros(40)
    zerorates = np.zeros(12)  
    for i in range(len(zeros)):
        zeros[i] = -1
        tzeros[i] = zeros[i]
    zerorates[0] = np.log(1+YTMs[0]*Maturity[0])/Maturity[0]
    zeros[0] = zerorates[0]
    tzeros[0] = zeros[0]
    def Par_value(seed,N) :
        tmpPrice = 0
        numbuck = int(Maturity[N]/0.25)           
        tzeros[numbuck-1] = seed
        for i in range(0,numbuck-1):            
            if zeros[i] < 0 :
                tzeros[i] = (0.25*tzeros[numbuck-1] + (Maturity[N]-0.25*(i+1))*tzeros[i-1])/(Maturity[N]-0.25*i)
                tmpPrice = tmpPrice + (YTMs[N]*0.25)*exp(-tzeros[i]*0.25*(i+1))
            else :
                tmpPrice = tmpPrice + (YTMs[N]*0.25)*exp(-zeros[i]*0.25*(i+1))    
        tmpPrice2 = tmpPrice + (1+YTMs[N]*0.25)*exp(-seed * Maturity[N])
        return tmpPrice2 - 1    
    for i in range(1,12):
        optimize.newton(Par_value , 0 , args = (i,) )
        zerorates[i] = optimize.newton(Par_value , 0 , args = (i,) ) 
        for j in range(0,40):
            zeros[j] = tzeros[j]
    b[r] = zeros
            #for i in range(0,12):
           #     a.loc[i,r] = zerorates[i]     
           # a = a.rename(columns={(r):data.loc[r,'일자']})
a = zerorates


# x = [b1,b2,b3,b4,a1,a2]
f = lambda k,M : (k[0]) + (k[1]*((1-exp(- M/k[4]))/(M/k[4]))) + (k[2]*(((1-exp(-M/k[4]))/(M/k[4]))-(exp(-M/k[4])))) + (k[3]*(((1-exp(-M/k[5]))/(M/k[5]))-(exp(-M/k[5]))))
def error(x,M,y_data) :
    if x[0]+x[1] > 0 and x[0] > 0 :
        return ((f(x,M)-y_data)**2).sum() 
    else :
        return 1000                              # NSS Model과 error func 함수 설정

M = array(Maturity)
y_data = array(zerorates)
x0 = array([0.01,0.01,0.01,0.01,1,1])                  #주어진 데이터 입력
x = optimize.fmin(error , x0 , args = (M,y_data,))     #error func의 최솟값
m = x
        
    
class MyWindow(QWidget) :
    def __init__(self) :
        super().__init__()
        
        self.table()
    
    def table(self) :
        self.setGeometry(800,200,1200,500)
        
        ##### Maturity YTMs의 표를 만든다.#####
        
        self.tableWidget = QTableWidget(self)
        self.tableWidget.setRowCount(12)
        self.tableWidget.setColumnCount(2)
        self.tableWidget.setEditTriggers(QAbstractItemView.NoEditTriggers)
        self.tableWidget.setHorizontalHeaderLabels(['Maturity' , 'YTMs'])
        
        for k,v in Bond_data.items() :
            col = column_idx_lookup[k]
            for row, val in enumerate(v) :
                item = QTableWidgetItem(val)
                
                self.tableWidget.setItem(row,col,item)
        
        self.tableWidget.resizeColumnsToContents()
        self.tableWidget.resizeRowsToContents()
                         
        ###### 버튼을 만든다 #####
        
        self.Button = QPushButton("Bootstrap",self)
        self.Button.clicked.connect(self.Button_clicked)
        
        self.Button1 = QPushButton("Interpolation and Extrapolation",self)
        self.Button1.clicked.connect(self.Button_clicked1)
        
        self.Button2 = QPushButton("NSS Model",self)
        self.Button2.clicked.connect(self.Button_clicked2)
                         
        ###### zerorates을 추가한 표를 만든다.#########
        
        self.tableWidget2 = QTableWidget(self)
        self.tableWidget2.setRowCount(12)
        self.tableWidget2.setColumnCount(3)
        self.tableWidget2.setHorizontalHeaderLabels(['Mat' , 'YTMs' , 'Zerorates'])
        
        for k,v in Bond_data.items() :
            col = column_idx_lookup[k]
            for row, val in enumerate(v) :
                item = QTableWidgetItem(val)
                
                self.tableWidget2.setItem(row,col,item)
       
        self.tableWidget2.resizeColumnsToContents()
        self.tableWidget2.resizeRowsToContents()                      
                     
        ###### Interpolation 표를 만든다   ######    
            
        self.tableWidget3 = QTableWidget(self)
        self.tableWidget3.setRowCount(120)
        self.tableWidget3.setColumnCount(2)
        self.tableWidget3.setHorizontalHeaderLabels(['Mat' , 'Zerorates'])
        
        for k,v in Bond_data.items() :
            col = 0
            for row in range(0,120) :
                i = '%s' % (row+1)
                item = QTableWidgetItem(i)
                self.tableWidget3.setItem(row,col,item)
       
        self.tableWidget3.resizeColumnsToContents()
        self.tableWidget3.resizeRowsToContents()   
            
        ####### NSS Fitting Parameter와 그래프를 만든다 ######
                                                                
        self.tableWidget4 = QTableWidget(self)
        self.tableWidget4.setRowCount(1)
        self.tableWidget4.setColumnCount(6)
        self.tableWidget4.setHorizontalHeaderLabels(['beta0','beta1','beta2','beta3','tau0','tau1'])
        
        self.tableWidget4.resizeColumnsToContents()
        self.tableWidget4.resizeRowsToContents() 
        
        
        self.Button3 = QPushButton("Graph",self)
        self.Button3.clicked.connect(self.Button_clicked3)
        
        self.fig = plt.Figure()
        self.canvas = FigureCanvas(self.fig)
        
        
                                                                 # 1번 위젯만들기
        ##### 레이아웃을 만든다 #####
        leftLayout = QVBoxLayout()                               # 2번 레이아웃 매니저 만들기
        leftLayout.addWidget(self.tableWidget)                  # 3번 addWidget메서드를 이용한 위젯등록
        leftLayout.addWidget(self.Button)
        
        rightLayout = QVBoxLayout()
        rightLayout.addWidget(self.tableWidget2)
        rightLayout.addWidget(self.Button1)
        rightLayout.addWidget(self.Button2)
        
        rightLayout2 = QVBoxLayout()
        rightLayout2.addWidget(self.tableWidget3)
        
        rightLayout3 = QVBoxLayout()
        rightLayout3.addWidget(self.tableWidget4)
        rightLayout3.addWidget(self.Button3)
        rightLayout3.addWidget(self.canvas)
        
        layout = QHBoxLayout()
        layout.addLayout(leftLayout)
        layout.addLayout(rightLayout)
        layout.addLayout(rightLayout2)
        layout.addLayout(rightLayout3)
      
        
        self.setLayout(layout)                                 # 4번 레이아웃 매니저 등록 단계
        

    def Button_clicked(self) :
        ############# 저장되어있는 zerorates를 Bond_data_2에 저장한다 ##########
        Bond_data_2 = {'Zerorates' : ["%s" %a[0],"%s" %a[1],"%s" %a[2],"%s" %a[3],"%s" %a[4],"%s" %a[5],"%s" %a[6],"%s" %a[7],"%s" %a[8],"%s" %a[9],"%s" %a[10],"%s" %a[11]]}
        for k,v in Bond_data_2.items():
            col = column_idx_lookup_2[k]
            for row,val in enumerate(v) :
                item = QTableWidgetItem(val)
                
                self.tableWidget2.setItem(row,col,item)
        self.tableWidget2.resizeColumnsToContents()
        self.tableWidget2.resizeRowsToContents()       
        
        
    def Button_clicked1(self) :
        # Interpolation 표를 채운다 
        x = np.linspace(3,120,40)
        y = np.array(b[r])
                                             #x_new = np.linspace(3,120,118)
                                             #f = interpolate.interp1d(x,y)
                                             #f(x_new)
        y_interp = scipy.interpolate.interp1d(x,y)
        for i in range(3,121):
            c.loc[i-1,r] = y_interp(i)
        data1 = np.array([[3,c.loc[2,r]],[4,c.loc[3,r]]])
        fit = np.polyfit(data1[:,0],data1[:,1],1)
        line = np.poly1d(fit)
        new_points = np.arange(2) + 1
        c.loc[0,r] = np.array( line(new_points)[0] )
        c.loc[1,r] = np.array( line(new_points)[1] )
            #for i in range(0,12):
           #     a.loc[i,r] = zerorates[i]     
           # a = a.rename(columns={(r):data.loc[r,'일자']})
            
        k = []
        for i in range(0,120) :
            k.append("%s" %c.loc[i,1])
        Bond_data_3 = {'Zerorates' : k}
        for k,v in Bond_data_3.items():
            col = 1
            for row,val in enumerate(v) :
                item = QTableWidgetItem(val)
                self.tableWidget3.setItem(row,col,item)
        self.tableWidget3.resizeColumnsToContents()
        self.tableWidget3.resizeRowsToContents()
        
    def Button_clicked2(self) :
        # NSS 표를 채운다
        
        n=[]
        for i in range(0,6):
            n.append("%s" %m[i])
        Nss_parameter = {'beta0':n[0],'beta1':n[1],'beta2':n[2],'beta3':n[3],'tau0':n[4],'tau1':n[5]}
        for k,v in Nss_parameter.items():
            row = 0
            for col in range(0,6) :
                item = QTableWidgetItem(n[col])
                
                self.tableWidget4.setItem(row,col,item)
        self.tableWidget4.resizeColumnsToContents()
        self.tableWidget4.resizeRowsToContents()       
    
        
    def Button_clicked3(self) : 
        #QMessageBox.about(self,"message","Graph")
        ax = self.fig.add_subplot(111)
        y = [f(m,v) for v in M]
        
        ax.plot(M,y)
        ax.scatter(M,y_data)
        
        
        self.canvas.draw()
        
if __name__ == "__main__" :
    app = QApplication(sys.argv)
    mywindow = MyWindow()
    mywindow.show()
    app.exec_()

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 480
         Function evaluations: 743
